# exercise 1: generate 3d conformation

In [1]:
from openbabel import pybel
import subprocess
# subprocess 모듈: python에서 쉘 명령어나 프로그램을 실행

In [2]:
run_line = 'ls'
subprocess.run(run_line.split())

1_gen3d.ipynb
2_pdbqt2pdb.ipynb
M855.pdb
M855.pdbqt
pdb


CompletedProcess(args=['ls'], returncode=0)

In [3]:
run_line = 'ls'
result = subprocess.run(run_line.split(),capture_output=True,
                        check=True, universal_newlines=True)
print(result)

CompletedProcess(args=['ls'], returncode=0, stdout='1_gen3d.ipynb\n2_pdbqt2pdb.ipynb\nM855.pdb\nM855.pdbqt\npdb\n', stderr='')


In [5]:
# neutralization
smi0 = 'C(=O)([O-])CCC(=[NH2+])N'
run_line = 'obabel -:%s -osmi --neutralize' % (smi0)
result = subprocess.run(run_line.split(), capture_output=True,
                        check=True, universal_newlines=True,)
print(result)
print(result.stdout)

CompletedProcess(args=['obabel', '-:C(=O)([O-])CCC(=[NH2+])N', '-osmi', '--neutralize'], returncode=0, stdout='C(=O)(O)CCC(=N)N\t\n', stderr='1 molecule converted\n')
C(=O)(O)CCC(=N)N	



In [6]:
# protonation state
pH = 7.4
smi0 = 'C(=O)(O)CCC(=N)N'
m = pybel.readstring("smi", smi0)
m.OBMol.AddHydrogens(False, True, pH)
smi_p = m.write("smi").strip()
print(smi_p)

C(=O)([O-])CCC(=[NH2+])N


In [7]:
# protonation state
pH = 7.4
smi0 = 'C(=O)(O)CCC(=N)N'
run_line = 'obabel -:%s -osmi -p %s' % (smi0, pH)
result = subprocess.run(run_line.split(), capture_output=True,
                        check=True, universal_newlines=True)
print(result)
print(result.stdout)

CompletedProcess(args=['obabel', '-:C(=O)(O)CCC(=N)N', '-osmi', '-p', '7.4'], returncode=0, stdout='C(=O)([O-])CCC(=[NH2+])N\t\n', stderr='1 molecule converted\n')
C(=O)([O-])CCC(=[NH2+])N	



In [8]:
smi = 'Cc1cccc(n1)c1nc(c2ccccc2n1)Nc1ccncc1'
timeout = 10
run_line = 'obabel -:%s --gen3D -opdb' % (smi)
result = subprocess.run(run_line.split(), capture_output=True,
                        check=True, universal_newlines=True,
                        timeout=timeout)
#print(result)

In [9]:
def check_gen3d_pdb(line_list):
    count_0 = 0
    check_error = False
    for line in line_list:
        if line[0:6] == 'HETATM' or line[0:6] == 'ATOM  ':
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])
            if x == 0.0 and y == 0.0 and z == 0.0:
                count_0 += 1
            if count_0 >= 2:
                check_error = True
                break

    return check_error

In [10]:
def fix_ligand_atom_idx(line_list):

    atom_dict = dict()
    total_line_out = str()

    for line in line_list:
        line = line.rstrip('\n')
        if line[0:6] == 'ATOM  ' or line[0:6] == 'HETATM':
            atom = line[12:16]
            at = atom[0:2]
            chain = line[21]
            line = 'HETATM%s%s   UNK %s   1 %s' % (line[6:12], at,
                                                   chain, line[27:])
        if line[0:6] == 'HETATM':
            atom = line[12:16]
            at = atom[0:2]
            if at not in atom_dict:
                atom_dict[at] = 0
#            chain = line[21]
            atom_dict[at] += 1
            idx = atom_dict[at]
            line_out = line[0:12] + '%s%-2d' % (at, idx) + line[16:]
        else:
            line_out = line
        total_line_out += line_out + '\n'

    return total_line_out

In [11]:
def gen_3d_pdb(smi, ligand_file, timeout=10):
    """
        generate initial 3d conformation from SMILES
        input :
            SMILES string
            ligand_file (output file, pdb)
    """
    run_line = 'obabel -:%s --gen3D -opdb' % (smi)
    e = None
    try:
        result = subprocess.run(run_line.split(), capture_output=True,
                                check=True, universal_newlines=True,
                                timeout=timeout)
    except Exception as e:
        return e

    err_lines = result.stderr.split('\n')
    for i, line in enumerate(err_lines):
        idx = line.find('Error')
        if idx != -1:
            e = err_lines[i] + err_lines[i+1]
            return e

    result_lines = result.stdout.strip('\n').split('\n')
    check_error = check_gen3d_pdb(result_lines)
    if check_error:
        e = 'error: gen 3d, two or more (0,0,0)'
        return e

    total_line_out = fix_ligand_atom_idx(result_lines)
    fp = open(ligand_file, 'w')
    fp.write(total_line_out)
    fp.close()
    
    return e

In [12]:
smi = 'Cc1cccc(n1)c1nc(c2ccccc2n1)Nc1ccncc1'
timeout = 10
ligand_file = 'M855.pdb'
gen_3d_pdb(smi, ligand_file, timeout=10)

In [13]:
def pdb_to_pdbqt(pdb_file, pdbqt_file):

    run_line = 'prepare_ligand4.py -l %s -o %s' % (pdb_file, pdbqt_file)
    run_line += ' -U nphs_lps'
    e = None
    try:
        subprocess.check_output(run_line.split(), stderr=subprocess.STDOUT,
                                universal_newlines=True)
    except Exception as e:
        return e
    return e

In [14]:
pdb_file = 'M855.pdb'
pdbqt_file = 'M855.pdbqt'
pdb_to_pdbqt(pdb_file, pdbqt_file)